In [ ]:
import os
import cv2
import torch
import gc
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import albumentations as A
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ==========================================
# 1. CONFIGURATION
# ==========================================
class Config:
    # Update these paths if your Kaggle directory structure is different
    ROOT = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle"
    TRAIN_IMG = os.path.join(ROOT, "train_images")
    TRAIN_MSK = os.path.join(ROOT, "train_masks")
    TEST_IMG  = os.path.join(ROOT, "test_images_padded")
    
    IMG_SIZE = (512, 512)
    VAL_SPLIT = 0.15        
    BATCH_SIZE = 4          
    ACCUMULATION_STEPS = 4  
    NUM_WORKERS = 4         
    LR = 6e-5               
    EPOCHS = 30             
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 'nvidia/mit-b5' is accurate but heavy. 
    # If you hit OOM (Out Of Memory), switch to 'nvidia/mit-b2' or 'nvidia/mit-b1'.
    MODEL_CHECKPOINT = "nvidia/mit-b5" 
    SAVE_PATH = "best_segformer_avl.pth"

# ==========================================
# 2. AVL: ADAPTIVE VALIDATION LOGIC
# ==========================================
class AVLTracker:
    """
    Adaptive Validation Logic.
    Tracks Exponential Moving Average (EMA) of IoU to prevent saving models 
    based on 'lucky' spikes in validation performance.
    """
    def __init__(self, window_size=3):
        self.window_size = window_size
        self.history = []
        self.best_ema_iou = 0.0

    def update(self, current_iou):
        self.history.append(current_iou)
        if len(self.history) > self.window_size:
            self.history.pop(0)
        
        # Calculate Moving Average
        ema_iou = sum(self.history) / len(self.history)
        
        is_best = False
        if ema_iou > self.best_ema_iou:
            self.best_ema_iou = ema_iou
            is_best = True
        
        return ema_iou, is_best

# ==========================================
# 3. TVERSKY LOSS
# ==========================================
class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.3, beta=0.7, smooth=1e-6):
        """
        Tversky Loss Implementation.
        
        Args:
            alpha (float): Weight for False Positives.
            beta (float):  Weight for False Negatives. 
                           If beta > alpha, we penalize missing the target more (Recall focused).
            smooth (float): Smoothing factor for numerical stability.
        """
        super(TverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.smooth = smooth

    def forward(self, logits, targets):
        # 1. Resize logits to match target size (upsampling predictions)
        # This preserves the detail in the ground truth masks.
        if logits.shape[-2:] != targets.shape[-2:]:
            logits = F.interpolate(logits, size=targets.shape[-2:], mode='bilinear', align_corners=False)
            
        # 2. Sigmoid activation
        probs = torch.sigmoid(logits)
        
        # 3. Flatten
        probs = probs.view(-1)
        targets = targets.view(-1)
        
        # 4. Calculate TP, FP, FN
        TP = (probs * targets).sum()
        FP = ((1 - targets) * probs).sum()
        FN = (targets * (1 - probs)).sum()
        
        # 5. Tversky Index
        tversky_index = (TP + self.smooth) / (TP + self.alpha * FP + self.beta * FN + self.smooth)
        
        return 1 - tversky_index

# ==========================================
# 4. DATASET
# ==========================================
class TerrainDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, file_list=None, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        
        if file_list is not None:
            self.images = file_list
        else:
            self.images = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg'))])

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        image_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.mask_dir:
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = (mask > 127).astype(np.float32)
            
            if self.transform:
                aug = self.transform(image=image, mask=mask)
                image = aug['image']
                mask = aug['mask']
            
            return image, mask.unsqueeze(0)
        
        h, w = image.shape[:2]
        if self.transform:
            aug = self.transform(image=image)
            image = aug['image']
            
        return image, img_name, (h, w)

# Helper for IoU Calculation
def calculate_iou(preds, targets):
    if preds.shape[-2:] != targets.shape[-2:]:
        preds = F.interpolate(preds, size=targets.shape[-2:], mode='bilinear', align_corners=False)
        
    preds = (torch.sigmoid(preds) > 0.5).float()
    intersection = (preds * targets).sum()
    union = (preds + targets).clamp(0, 1).sum()
    return (intersection / (union + 1e-7)).item()

# ==========================================
# 5. TRAINING LOOP
# ==========================================
def train_model():
    torch.cuda.empty_cache()
    gc.collect()

    # 5.1 Data Prep
    all_images = sorted([f for f in os.listdir(Config.TRAIN_IMG) if f.lower().endswith(('.png', '.jpg'))])
    train_files, val_files = train_test_split(all_images, test_size=Config.VAL_SPLIT, random_state=42)

    transforms = {
        "train": A.Compose([
            A.Resize(Config.IMG_SIZE[0], Config.IMG_SIZE[1]),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.2),
            A.Normalize(),
            ToTensorV2()
        ]),
        "val": A.Compose([
            A.Resize(Config.IMG_SIZE[0], Config.IMG_SIZE[1]),
            A.Normalize(),
            ToTensorV2()
        ])
    }

    train_loader = DataLoader(
        TerrainDataset(Config.TRAIN_IMG, Config.TRAIN_MSK, train_files, transforms["train"]), 
        batch_size=Config.BATCH_SIZE, shuffle=True, num_workers=Config.NUM_WORKERS, pin_memory=True
    )
    val_loader = DataLoader(
        TerrainDataset(Config.TRAIN_IMG, Config.TRAIN_MSK, val_files, transforms["val"]), 
        batch_size=Config.BATCH_SIZE, num_workers=Config.NUM_WORKERS, pin_memory=True
    )

    # 5.2 Model Setup
    print(f"🏗️ Loading Model: {Config.MODEL_CHECKPOINT}...")
    model = SegformerForSemanticSegmentation.from_pretrained(
        Config.MODEL_CHECKPOINT, 
        num_labels=1, 
        ignore_mismatched_sizes=True
    ).to(Config.DEVICE)
    
    optimizer = optim.AdamW(model.parameters(), lr=Config.LR, weight_decay=1e-2)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3) 
    scaler = GradScaler()
    
    # 5.3 Loss Definition (USING TVERSKY)
    # Alpha=0.3, Beta=0.7 favors Recall (penalizes missing obstacles more than false alarms)
    criterion = TverskyLoss(alpha=0.3, beta=0.7)
    
    # AVL Initialize
    avl = AVLTracker(window_size=3)

    print(f"🚀 Training Started with Tversky Loss...")

    for epoch in range(Config.EPOCHS):
        model.train()
        train_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{Config.EPOCHS} [TRAIN]")
        
        for i, (imgs, msks) in enumerate(pbar):
            imgs, msks = imgs.to(Config.DEVICE), msks.to(Config.DEVICE)
            
            with autocast():
                outputs = model(imgs)
                # Note: outputs.logits are typically 1/4th size, criterion handles interpolation
                loss = criterion(outputs.logits, msks) / Config.ACCUMULATION_STEPS
            
            scaler.scale(loss).backward()
            
            if (i + 1) % Config.ACCUMULATION_STEPS == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            train_loss += loss.item() * Config.ACCUMULATION_STEPS
            pbar.set_postfix(loss=train_loss/(i+1))

        # VALIDATION
        model.eval()
        val_iou = 0
        with torch.no_grad():
            for imgs, msks in val_loader:
                imgs, msks = imgs.to(Config.DEVICE), msks.to(Config.DEVICE)
                val_iou += calculate_iou(model(imgs).logits, msks)
        
        avg_iou = val_iou / len(val_loader)
        
        # AVL LOGIC CHECK
        ema_iou, is_best = avl.update(avg_iou)
        scheduler.step(ema_iou)
        
        print(f"📊 Epoch {epoch+1} | Raw IoU: {avg_iou:.4f} | AVL (EMA) IoU: {ema_iou:.4f}")

        if is_best:
            torch.save(model.state_dict(), Config.SAVE_PATH)
            print(f"⭐ AVL System: New stable high score detected. Model saved!")

# ==========================================
# 6. INFERENCE
# ==========================================
def run_inference():
    print("🔮 Starting Inference...")
    model = SegformerForSemanticSegmentation.from_pretrained(
        Config.MODEL_CHECKPOINT, 
        num_labels=1, 
        ignore_mismatched_sizes=True
    ).to(Config.DEVICE)
    
    model.load_state_dict(torch.load(Config.SAVE_PATH))
    model.eval()

    test_ds = TerrainDataset(Config.TEST_IMG, transform=A.Compose([
        A.Resize(Config.IMG_SIZE[0], Config.IMG_SIZE[1]), 
        A.Normalize(), 
        ToTensorV2()
    ]))
    loader = DataLoader(test_ds, batch_size=1)
    
    results = []
    
    # Run-Length Encoding Helper
    def rle(m):
        p = m.T.flatten(); p = np.concatenate([[0], p, [0]])
        r = np.where(p[1:] != p[:-1])[0] + 1; r[1::2] -= r[::2]
        return ' '.join(str(x) for x in r)

    with torch.no_grad():
        for img, name, (h, w) in tqdm(loader, desc="Inference"):
            logits = model(img.to(Config.DEVICE)).logits
            # Resize logits to original image size
            logits = F.interpolate(logits, size=(h.item(), w.item()), mode='bilinear')
            mask = (torch.sigmoid(logits) > 0.5).cpu().numpy()[0, 0].astype(np.uint8)
            results.append({"image_id": int(os.path.splitext(name[0])[0]), "encoded_pixels": rle(mask)})

    pd.DataFrame(results).sort_values("image_id").to_csv("submission.csv", index=False)
    print("✨ submission.csv saved via AVL optimization & Tversky Loss!")

if __name__ == "__main__":
    train_model()
    run_inference()